In [1]:
from lightautoml.dataset.seq_np_pd_dataset import SeqNumpyPandasDataset
from lightautoml.reader.seq import IDSInd, TopInd
from lightautoml.reader.base import DictToPandasSeqReader
from lightautoml.tasks import Task
import pandas as pd
import numpy as np
from lightautoml.automl.base import AutoML
from lightautoml.ml_algo.boost_lgbm import BoostLGBM

from lightautoml.pipelines.features.lgb_pipeline import LGBSimpleFeatures, LGBSeqSimpleFeatures, LGBMultiSeqSimpleFeatures
from lightautoml.pipelines.ml.base import MLPipeline

import numpy as np

In [2]:
import pandas as pd
import numpy as np

DATA_FOLDER = '../data/relational_data'

fulfilment_center_info = pd.read_csv(f'{DATA_FOLDER}/fulfilment_center_info.csv')
meal_info = pd.read_csv(f'{DATA_FOLDER}/meal_info.csv')
train = pd.read_csv(f'{DATA_FOLDER}/train.csv')

val = 0.05


In [3]:
seq_params = {
              'fulfilment_center_info':{'case': 'ids',
                       'params': {},
                       'scheme': {'to': 'plain', 'from_id': 'center_id', 'to_id': 'center_id'},
                          },
              'meal_info':{'case': 'ids',
                       'params': {},
                       'scheme': {'to': 'plain', 'from_id': 'meal_id', 'to_id': 'meal_id'},
                          },
              }
seq_data = {'fulfilment_center_info': fulfilment_center_info[['center_id',
                                                              'city_code',
                                                              'region_code',
                                                              'op_area']],
       'meal_info': meal_info              
       }

X_train = {'plain':train.sample(frac=0.05).reset_index(drop=True) , 
           'seq': seq_data
          }

task = Task('reg', metric='mae')

roles={'target': 'num_orders'}




In [4]:
task = Task('reg', metric='mae')
roles={'target': 'num_orders'}
reader = DictToPandasSeqReader(task=task, seq_params=seq_params)

feats = LGBMultiSeqSimpleFeatures()
model = BoostLGBM()
pipeline_lvl1 = MLPipeline([model], pre_selection=None, features_pipeline=feats, post_selection=None)

automl = AutoML(reader, [
    [pipeline_lvl1],
], skip_conn=False)

In [5]:
%%time

oof_pred = automl.fit_predict(X_train, roles=roles, verbose=3)


Feats was rejected during automatic roles guess: []
[00:42:35] Layer 1 train process start. Time left 9999999996.70 secs
[00:42:36] Start fitting Lvl_0_Pipe_0_Mod_0_LightGBM ...
[00:42:36] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LightGBM =====
[00:42:36] Training until validation scores don't improve for 100 rounds
[00:42:37] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_LightGBM =====
[00:42:37] Training until validation scores don't improve for 100 rounds
[00:42:38] ===== Start working with fold 2 for Lvl_0_Pipe_0_Mod_0_LightGBM =====
[00:42:38] Training until validation scores don't improve for 100 rounds
[00:42:39] ===== Start working with fold 3 for Lvl_0_Pipe_0_Mod_0_LightGBM =====
[00:42:39] Training until validation scores don't improve for 100 rounds
[00:42:40] ===== Start working with fold 4 for Lvl_0_Pipe_0_Mod_0_LightGBM =====
[00:42:40] Training until validation scores don't improve for 100 rounds
[00:42:41] Fitting Lvl_0_Pipe_0_Mod_0_LightGBM finis

In [6]:
from lightautoml.dataset.gpu.gpu_dataset import SeqCudfDataset
from lightautoml.dataset.gpu.gpu_dataset import SeqDaskCudfDataset
from lightautoml.reader.gpu.seq_gpu import IDSInd_gpu, TopInd_gpu
from lightautoml.reader.gpu.seq_reader_gpu import DictToCudfSeqReader
from lightautoml.reader.gpu.seq_reader_gpu import DictToDaskCudfSeqReader

from lightautoml.ml_algo.gpu.boost_xgb_gpu import BoostXGB
from lightautoml.ml_algo.gpu.boost_cb_gpu import BoostCB_gpu

from lightautoml.pipelines.features.gpu.lgb_pipeline_gpu import LGBMultiSeqSimpleFeatures_gpu

import cudf
import dask_cudf
import cupy as cp

In [7]:
task = Task('reg', metric='mae', device='gpu')
reader = DictToCudfSeqReader(task=task, seq_params=seq_params, n_jobs=1)

feats = LGBMultiSeqSimpleFeatures_gpu()
model = BoostCB_gpu(gpu_ids=[0])
pipeline_lvl1 = MLPipeline([model], pre_selection=None, features_pipeline=feats, post_selection=None)

automl = AutoML(reader, [
    [pipeline_lvl1],
], skip_conn=False)

In [8]:
oof_pred_gpu = automl.fit_predict(X_train, roles=roles, verbose=3)

[00:42:46] Feats was rejected during automatic roles guess: []
[00:42:46] Layer 1 train process start. Time left 9999999998.36 secs
[00:42:46] Start fitting Lvl_0_Pipe_0_Mod_0_CatBoost_gpu ...
[00:42:46] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_CatBoost_gpu (orig) =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
Default metric period is 5 because MAE is/are not implemented for GPU


[00:42:52] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_CatBoost_gpu (orig) =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
Default metric period is 5 because MAE is/are not implemented for GPU


[00:42:57] ===== Start working with fold 2 for Lvl_0_Pipe_0_Mod_0_CatBoost_gpu (orig) =====


...

[00:43:01] ===== Start working with fold 3 for Lvl_0_Pipe_0_Mod_0_CatBoost_gpu (orig) =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
Default metric period is 5 because MAE is/are not implemented for GPU


[00:43:05] ===== Start working with fold 4 for Lvl_0_Pipe_0_Mod_0_CatBoost_gpu (orig) =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
Default metric period is 5 because MAE is/are not implemented for GPU


[00:43:09] Lvl_0_Pipe_0_Mod_0_CatBoost_gpu fitting and predicting completed
[00:43:09] Time left 9999999975.27 secs

[00:43:09] Layer 1 training completed.



In [9]:
task = Task('reg', metric='mae', device='mgpu')
reader = DictToDaskCudfSeqReader(task=task, seq_params=seq_params, n_jobs=1)

feats = LGBMultiSeqSimpleFeatures_gpu()
model = BoostCB_gpu(gpu_ids=[0])
pipeline_lvl1 = MLPipeline([model], pre_selection=None, features_pipeline=feats, post_selection=None)

automl = AutoML(reader, [
    [pipeline_lvl1],
], skip_conn=False)

In [10]:
oof_pred_mgpu = automl.fit_predict(X_train, roles=roles, verbose=3)

[00:43:10] Feats was rejected during automatic roles guess: []
[00:43:11] Layer 1 train process start. Time left 9999999997.95 secs


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/dask/dataframe/core.py:4948: UserWarning: New index has same name as existing, this is a no-op.
  warnings.warn(
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/dask/dataframe/core.py:4948: UserWarning: New index has same name as existing, this is a no-op.
  warnings.warn(


[00:43:12] Start fitting Lvl_0_Pipe_0_Mod_0_CatBoost_gpu ...
[00:43:12] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_CatBoost_gpu (orig) =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
Default metric period is 5 because MAE is/are not implemented for GPU


[00:43:17] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_CatBoost_gpu (orig) =====


...

[00:43:25] ===== Start working with fold 3 for Lvl_0_Pipe_0_Mod_0_CatBoost_gpu (orig) =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
Default metric period is 5 because MAE is/are not implemented for GPU


[00:43:30] ===== Start working with fold 4 for Lvl_0_Pipe_0_Mod_0_CatBoost_gpu (orig) =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
Default metric period is 5 because MAE is/are not implemented for GPU


[00:43:33] Lvl_0_Pipe_0_Mod_0_CatBoost_gpu fitting and predicting completed
[00:43:33] Time left 9999999975.94 secs

[00:43:33] Layer 1 training completed.

